### Import packages to graph results

In [11]:
from glob import glob
import pandas as pd
import json
#import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'iframe'
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# Used for debug
# from pprint import pprint

### Set default color palete and size for making plots

In [12]:
colorPallet = ["rgba(151, 209, 233, 255)", "rgba(0, 120, 179, 255)", "rgba(179, 223, 146, 255)", "rgba(49, 169, 90, 255)", "rgba(227, 136, 220, 255)", "rgba(127, 0, 255, 255)", "rgba(255, 128, 0, 255)"]
bgColor = 'rgba(245,245,255,255)'

baseFigureWidth = 700
baseFigureHeight = 350

### glob all the log files into a list, then create a dataframe with all json data

In [13]:
def create_dataframe(logs):
    logs = glob(logs)
    df_list = []
    for log in logs:
        with open(log) as f:
            obj = json.load(f)['saaf']
        # cpuInfo is messing up the dataframe... remove for now
        obj.pop('cpuInfo')
        df_list.append(pd.DataFrame([obj]))
    return pd.concat(df_list, ignore_index=True)

In [14]:
normal_df = create_dataframe("../logs/openfaas/kops_multi-tenant_spot_1x_8vcpu_16gb/bwa/*normal*")

In [15]:
tumor_df = create_dataframe("../logs/openfaas/kops_multi-tenant_spot_1x_8vcpu_16gb/bwa/*tumor*")

# Make bar plots to profile bwa function data

In [16]:
fig = make_subplots(specs = [[{"secondary_y": False}]])

# Add figure title
fig.update_layout(
    barmode='stack',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.47
    ),
    margin=dict(
        t=0,
        b=1,
        l=1,
        r=1,
        autoexpand=True
    ),
    font=dict(
        size=16
    ),
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    plot_bgcolor=bgColor
)

profileData = pd.DataFrame()
profileData['payload'] = ["Normal", "Tumor"]
# Metrics for profiling
metrics = ['SoftIrq', 'Kernel', 'IOWait', 'User', 'Idle']
profileData['cpuTotalTime'] = [0, 0]
for metric in metrics:
    normalMean = normal_df[f'cpu{metric}Delta'].mean()
    tumorMean = tumor_df[f'cpu{metric}Delta'].mean()
    profileData[metric] = [normalMean, tumorMean]
    profileData['cpuTotalTime'] = [
        normalMean + profileData['cpuTotalTime'][0],
        tumorMean + profileData['cpuTotalTime'][1]
    ]

#profileData['runtime'] = [round(normal_df['runtime'].mean() / 1000), round(tumor_df['runtime'].mean() / 1000)]
#profileData['runtimeText'] = [f"Runtime:<br>{value} s" for value in profileData['runtime']]
for i, metric in enumerate(metrics):
    profileData[metric] = [
        profileData[metric][0] * 100 / profileData['cpuTotalTime'][0],
        profileData[metric][1] * 100 / profileData['cpuTotalTime'][1]
    ]
    text = [f'{round(val, 2)}%' for val in profileData[metric]]
    fig.add_trace(go.Bar(x = profileData["payload"],
                         y = profileData[metric],
                         name = f"CPU {metric}", marker_color=colorPallet[i],
                         text = text),
                  secondary_y=False)

# Set x-axis title
fig.update_xaxes(title_text="Payloads")

# Set y-axes titles
fig.update_yaxes(title_text="CPU Time Percentage", secondary_y=False)
# fig.update_yaxes(title_text="Function Runtime (ms)", secondary_y=True)

# py.iplot(fig)

fig.show()
# Save graph
# fig.write_image("graphs/bwa_profile_bars.pdf", width=baseFigureWidth, engine="kaleido", height=baseFigureHeight)